# 线性回归的简洁实现

### 生成数据集

In [8]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)


### 读取数据

In [9]:
import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

for X, y in data_iter:
    print(X, y)
    break

tensor([[ 0.1968, -0.2827],
        [ 0.7933,  2.2307],
        [-0.0220, -1.0971],
        [-2.7252, -1.5277],
        [ 0.7826, -0.5238],
        [-0.1910, -1.0743],
        [-1.0630,  1.7025],
        [ 0.0126,  0.3838],
        [ 1.5667,  0.7444],
        [ 0.8418,  0.2901]]) tensor([ 5.5614, -1.8043,  7.8765,  3.9540,  7.5404,  7.4679, -3.7214,  2.9358,
         4.7901,  4.8996])


### 定义模型

In [18]:
from torch import nn
'''
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    # 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net)'''

# 比较方便的定义模型
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))  # 模型输出为标量
print(net)

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [19]:
# 可以通过net.parameters()来查看模型所有的可学习参数，此函数将返回一个生成器
for param in net.parameters():
    print(param)


Parameter containing:
tensor([[-0.4415, -0.0589]], requires_grad=True)
Parameter containing:
tensor([-0.4474], requires_grad=True)


### 初始化模型参数

In [20]:
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01) # 均值为0，标准差为0.01
init.constant_(net[0].bias, val=0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)


Parameter containing:
tensor([0.], requires_grad=True)

### 定义损失函数

In [21]:
loss = nn.MSELoss()  # 均方误差

### 定义优化算法

In [26]:
import torch.optim as optim

# 是一个优化器
optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)


SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [25]:
'''# 调整学习率
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1 # 学习率为之前的0.1倍
print(optimizer) '''

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.00030000000000000003
    momentum: 0
    nesterov: False
    weight_decay: 0
)


### 训练模型

In [30]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))   # 输出为1维
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()      # 小批量的损失对模型参数求梯度
        optimizer.step()  # 迭代模型参数
    print('epoch %d, loss: %f' % (epoch, l.item()))


epoch 1, loss: 0.000078
epoch 2, loss: 0.000297
epoch 3, loss: 0.000058


In [28]:
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)


[2, -3.4] Parameter containing:
tensor([[ 1.9999, -3.4000]], requires_grad=True)
4.2 Parameter containing:
tensor([4.2000], requires_grad=True)
